실습) 영화제목 입력 받아 해당 영화의 평가를 알아보자.
- 평점
- 리뷰

In [57]:
# 모든 모듈 열기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import math
import os
import pandas as pd
import pyautogui
import numpy as np

# 제목
print("영화 평가 알아보기")
print()

# 입력 받기
what_movie = input("영화 제목을 입력하세요. ")
cnt = int(input("출력하고자 하는 리뷰 갯수를 입력하세요. "))

# IMDB 창 열기
from selenium import webdriver
chrome_path = 'E:\\py_temp\\WebCrawling\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
url = 'https://www.imdb.com/'
driver.get(url)
driver.maximize_window()
import time
time.sleep(2)

# 쿠키 삭제하기
driver.delete_all_cookies()
time.sleep(1)

# 입력받은 영화 검색하기 들어가기
search_bar = driver.find_element_by_xpath('//*[@id="suggestion-search"]')
search_bar.click()
search_bar.send_keys(what_movie)
search_bar.send_keys("\n")
movie_click = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a')
movie_click.click()

# 파일 생성 및 저장할 위치 지정하기
folder_path = 'E:\\py_temp\\WebCrawling\\%s' %("IMDB"+"_"+what_movie)
if not os.path.isdir(folder_path):
    os.makedirs(folder_path)
os.chdir(folder_path)

# 현재 시간 및 이름 지정하기
# 현재시간
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
# 이름 지정
text_name = "IMDB"+"_"+what_movie+"_"+s+".txt"
xlsx_name = "IMDB"+"_"+what_movie+"_"+s+".xlsx"
csv_name = "IMDB"+"_"+what_movie+"_"+s+".csv"

# 스크럴 다운하기
import pyautogui
pyautogui.scroll(4000)
time.sleep(3)

영화 평가 알아보기

영화 제목을 입력하세요. Parasite
출력하고자 하는 리뷰 갯수를 입력하세요. 77


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12756/2705660722.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12756/2705660722.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="suggestion-search"]')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12756/2705660722.py:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  movie_click = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a')


In [94]:
# 크롤링하기
f = open(text_name, 'a',encoding='UTF-8')

t_review = []
n_review = []
s_review = []
r_review = []

full_html = driver.page_source
soup = BeautifulSoup(full_html,'html.parser')
main = soup.find('main','ipc-page-wrapper ipc-page-wrapper--base')

print("1. Title:", main.find('h1','TitleHeader__TitleText-sc-1wu6n3d-0 dxSWFG').text.strip())
f.write("1. Title: %s \n" %main.find('h1','TitleHeader__TitleText-sc-1wu6n3d-0 dxSWFG').text.strip())

print("2. IMDB Rating:", main.find('div','AggregateRatingButton__Rating-sc-1ll29m0-2 bmbYRW').text.strip())
f.write("2. IMDB Rating: %s \n" %main.find('div','AggregateRatingButton__Rating-sc-1ll29m0-2 bmbYRW').text.strip())

print("3. Summary:", main.find('span','GenresAndPlot__TextContainerBreakpointXL-cum89p-2 gCtawA').text.strip())
f.write("3. Summary: %s \n" %main.find('span','GenresAndPlot__TextContainerBreakpointXL-cum89p-2 gCtawA').text.strip())

makers = main.find_all('ul',class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt")
print("4. Director:", makers[0].text.strip())
f.write("4. Director: %s \n" %makers[0].text.strip())
print("5. Writers:", makers[1].text.strip())
f.write("5. Writers: %s \n" %makers[1].text.strip())
print("6. Stars:", makers[2].text.strip())
f.write("6. Stars: %s \n" %makers[2].text.strip())

click_review = driver.find_element_by_link_text('User reviews')
click_review.click()

load = int(np.ceil(cnt/25))-1
load_more = driver.find_element_by_id('load-more-trigger')
import pyautogui
for x in range(0,int(load)):
    pyautogui.scroll(3000)
    time.sleep(4)
    load_more.click()
time.sleep(1)

full_html = driver.page_source
soup = BeautifulSoup(full_html,'html.parser')
print("7. Total Review Number:", soup.find('div','header').find('span').text.strip())
f.write("7. Total Review Number: %s \n" %soup.find('div','header').find('span').text.strip())
print("8. Reviews:")
box = soup.find('div','lister-list').find_all('div','review-container')

time.sleep(2)

for i in range(0,cnt):
    box_num = box[i]
    try:
        title = box_num.find('a','title').text.strip()
        t_review.append(title)
        try:
            star = box_num.find('span','rating-other-user-rating').find('span').text.strip()
        except:
            star = "none"
        s_review.append("%s/10" %star)
        name = box_num.find('span','display-name-link').text.strip()
        n_review.append(name)
        review = box_num.find('div','content').find('div').text.strip()
        r_review.append(review)
        print("> Review", (i+1), ":", "\n", "- title:", title, "\n", "- reviewer name:", name, "\n", "- reviewer rating:", star+"/10", "\n", "- review:", review)
        f.write("> Review: %s \n -title: %s \n -reviewer name: %s \n -reviewer rating: %s/10 \n -review: %s"%((i+1),title,name,star,review))
    except:
        print("> Review %s: No Review" %(i+1))
        f.write("> Review %s: No Review" %(i+1))
        pass
    print('-'*70)
    f.write('-'*70)
    f.write("\n")
    time.sleep(0.5)

1. Title: Gisaengchung
2. IMDB Rating: 8.6/10
3. Summary: Greed and class discrimination threaten the newly formed symbiotic relationship between the wealthy Park family and the destitute Kim clan.
4. Director: Bong Joon Ho
5. Writers: Bong Joon Ho(story by)Jin-won Han(screenplay by)
6. Stars: Kang-ho SongSun-kyun LeeYeo-jeong Cho


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12756/282585869.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  click_review = driver.find_element_by_link_text('User reviews')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12756/282585869.py:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  load_more = driver.find_element_by_id('load-more-trigger')


7. Total Review Number: 3,326 Reviews
8. Reviews:
> Review 1 : 
 - title: I loved the first 90%....but the last might be very tough for some viewers. 
 - reviewer name: MartinHafer 
 - reviewer rating: 6/10 
 - review: I was able to see "Parasite" a few days ago at the Philadelphia Film Festival and many people there loved the film....as did my daughter who saw it with me. As for me, I think the production values were excellent but the ending changed my overall feelings about the movie. Up untl the end, I liked it very, very much....but after the ultra-violent climax, I was left a bit cold. Your experience with the film is really dependent on whether or not you can take the violence and how it impacts you.Instead of talking about the plot and how great or not the film is, I want to focus on the violence. My daughter felt the violence wasn't that extreme and it was needed for the plot. As for me, seeing knives plunged into people at close range and seeing a giant rock smashed ultra-real

> Review 6 : 
 - title: Parasite 
 - reviewer name: Prismark10 
 - reviewer rating: 7/10 
 - review: Parasite became the first foreign language film to win the Best Picture Oscar. The first third of the film had me wondering why.The film opens in a dilapidated, smelly semi basement in South Korea. The Kim family is so poor that they need to borrow a neighbour's wifi and lose it when it becomes password protected. They make a living by folding cardboard pizza boxes.Dad, Ki-taek is the weak link. He cannot even fold the boxes properly so their meagre wages are deducted. Son, Ki-woo gets a stroke of fortune, when a friend helps him get a well paid tutoring job with a fake diploma. Ki-woo tutors the daughter of of the wealthy Park family who live in a lavish mansion. Soon Ki-woo gets his sister Ki-jung a job as an art tutor for the disruptive younger son.The wily Ki-jung conspires to get the Park's family chauffeur fired and have him replaced by their father. Before long they manoeuvre to 

> Review 12 : 
 - title: "If you make a plan, life never works out that way." 
 - reviewer name: classicsoncall 
 - reviewer rating: 8/10 
 - review: This film's ranking here on IMDb is somewhat of an anomaly. Usually, when a hot new release makes it's way into the Top 250, it manages to remain in it's general range for a while and then slowly decline as more and more reviewers add their vote. "Parasite" entered the list in the mid Fifties if I correctly recall, and since then has managed to improve it's position to #26 as I write this. No one knows if that will hold obviously, so I'm curious to see how the movie rates over time.Having stated that, I thought the film was brilliant in the way it opened and had the Kim family maneuver it's way into the life and home of the Park family. What was questionable to me was why the Kim's weren't able to improve their condition in life prior to the break they received when Min (Seo-joon Park) suggested the tutoring job to Ki-woo (Woo-sik Choi). 

...

> Review 51 : 
 - title: Powerful 
 - reviewer name: gbill-74877 
 - reviewer rating: 9/10 
 - review: Such a powerful allegory for the increasing wealth gap in the world. The scenes in the basement were chilling, especially that first one. The poor scurrying like cockroaches when the lights come on, swimming in their own filth during that flood scene, and turning against one another ... it was all devastating commentary on the position they're in, and the struggle. The rich guy who seems nice enough, but just the mere smell of the poor, or them not staying in their place and crossing the line he has in his mind which makes him quickly remind them of their relative positions ... it's a fantastic characterization, and I loved how it all built up into a crescendo with the opulence of that kid's birthday party. This is a message film that works on both levels, as it's hard to know how it will play out, and while dark, it's entertaining. The ending is brilliant too.
-----------------------

> Review 58 : 
 - title: Amazing, brilliant and just one of the few very best in 2019 
 - reviewer name: 851222 
 - reviewer rating: 10/10 
 - review: Greetings from Lithuania.Gisaengchung (2019) ("Parasite") is to simply put - the best South Korea'n movie i have ever seen. And i saw more then few of them and really loved them - i thing that this country makes best movies in Asia - in my personal opinion."Gisaengchung" is nothing short of a small masterpiece. It is a stupendously entertaining movie that "hooked" me from the first 45 seconds till the last frame for its entire 2 h 9 min running time - i was blown away of how amazingly crafted, acted, written and directed this movie was in every scene. The story was fresh and unique and universal. You can throw a rock at me if Hollywood won't remake this movie with some very well known actors in near future - it is a pure diamond in a movie market.Overall, "Gisaengchung" is by far one of the few best movies I've seen this year, right here

> Review 66 : 
 - title: Wow, just wow! 
 - reviewer name: paulclaassen 
 - reviewer rating: 8/10 
 - review: 'Parasite' can be summarized in a single word: WOW!Director Joon-ho Bong brought us the incredible films 'Memories of Murder', 'The Host', 'Mother', 'Snowpiercer' and 'Okja', and now 'Parasite'. Starting out as a comedy, the film soon reveals itself for what it truly is. This brilliant script has an ingenious set-up, morphing into a full-on, devious thriller.Initially innocent, the antagonist family's plan soon becomes evident as they infiltrate the lives of an unsuspecting wealthy family. At first through subtle, seemingly innocent actions, but becoming increasingly more sinister as their plans fall into place. The film has a few interesting twists. Kang-ho Song is excellent as usual. He is an incredibly versatile actor. I've seen him in comedies, action movies, horrors and thrillers and he simply never disappoints.The setting in the big mansion is really awesome, versus the d

> Review 71 : 
 - title: Good Enough 
 - reviewer name: pennyelenabooks 
 - reviewer rating: 8/10 
 - review: There is a big hype about this movie, and for a good reason. Unlike those popular films that those days create a buzz because of the action plot and the big names, Parasite is more about the social message that it wants to pass through than a likable story that will appear to the masses.So, overall, Parasite is good. If you like movies about real people, that are not afraid to show the raw reality and social injustice, than watch this one. It is a roller coaster of emotions, with lighter, more comical moments and more thrilling, suspenceful moments. And the film combines them masterfully and everything works well together. The performances were top notch as well. The story, finally, was well paced and interesting.
----------------------------------------------------------------------
> Review 72 : 
 - title: Started off great, then tanked - and I get the message 
 - reviewer na

> Review 75 : 
 - title: Parasite leeches onto societal dissociation to create a thrillingly hilarious satire on capitalism. 
 - reviewer name: TheMovieDiorama 
 - reviewer rating: 9/10 
 - review: "Rich people are naive. No resentments. No creases on them.". "It all gets ironed out. Money is an iron. Those creases all get smoothed out by money.". The Kim family, residing in the secluded darkness of a semi-underground basement apartment, acquire limited income. Jaggedly folding pizza boxes for an independent takeaway chain, with the already low-paying income reduced as a consequence of poor workmanship. They, much like the bottom-feeders of South Korea's working class division, struggle to survive. Consuming rotting food. Hijacking neighbouring paid Wi-Fi services. Residing in potentially diseased sub-urban streets. Ultimately aspiring to achieve the lofty heights of upper-class formalities. Conceptualising the popularised term "Hell Joseon", a critique on the current socioeconomic dec

In [95]:
# txt 파일에 크롤링 요약 정보 저장하기
import sys
orig_stdout = sys.stdout
f = open(text_name, 'a',encoding='UTF-8')
sys.stdout = f
sys.stdout = orig_stdout
f.close( )

# 번호 list 만들기
a_n = []
for i in range(1,cnt+1):
    a_n.append(i)

# EXCEL과 CSV 파일에 추출한 내용 넣기
f_movie = pd.DataFrame()
f_movie['Review Number']=pd.Series(a_n)
f_movie['Title']=pd.Series(t_review)
f_movie['Reviewer Rating']=pd.Series(s_review)
f_movie['Review']=pd.Series(r_review)
f_movie.to_csv(csv_name,encoding="utf-8-sig",index=False)
f_movie.to_excel(xlsx_name,index=False)

실습) TOP 10 영화 크롤링하기

In [4]:
# 모든 모듈 열기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import math
import os
import pandas as pd
import numpy as np

# IMDB 창 열기
from selenium import webdriver
chrome_path = 'E:\\py_temp\\WebCrawling\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
url = 'https://www.imdb.com/'
driver.get(url)
driver.maximize_window()
import time
time.sleep(2)

# 쿠키 삭제하기
driver.delete_all_cookies()
time.sleep(1)

# TOP 10 영화 창 들어가기
menu = driver.find_element_by_xpath('//*[@id="imdbHeader-navDrawerOpen--desktop"]')
menu.click()
time.sleep(0.5)
top10 = driver.find_element_by_xpath('//*[@id="imdbHeader"]/div[2]/aside/div/div[2]/div/div[1]/span/div/div/ul/a[6]')
top10.click()

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14776/1959268587.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14776/1959268587.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  menu = driver.find_element_by_xpath('//*[@id="imdbHeader-navDrawerOpen--desktop"]')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14776/1959268587.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  top10 = driver.find_element_by_xpath('//*[@id="imdbHeader"]/div[2]/aside/div/div[2]/div/div[1]/span/div/div/ul/a[6]')


In [55]:
# 각 영화 상세 페이지에 들어가 관련 정보 추출
def get():
    print()
    print("# TOP:",i)
    print()
    full_html = driver.page_source
    soup = BeautifulSoup(full_html,'html.parser')
    main = soup.find('main','ipc-page-wrapper ipc-page-wrapper--base')
    print("1. Title:", main.find('h1','TitleHeader__TitleText-sc-1wu6n3d-0 dxSWFG').text.strip())
    print("2. IMDB Rating:", main.find('div','AggregateRatingButton__Rating-sc-1ll29m0-2 bmbYRW').text.strip())
    print("3. Summary:", main.find('span','GenresAndPlot__TextContainerBreakpointXL-cum89p-2 gCtawA').text.strip())
    makers = main.find_all('ul',class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt")
    print("4. Director:", makers[0].text.strip())
    print("5. Writers:", makers[1].text.strip())
    print("6. Stars:", makers[2].text.strip())
    print("7. Reviewers:", main.find('span','ipc-title__subtext').text.strip())
    title = main.find('span','UserReviewSummary__Summary-kfza1v-1 bA-dHai').text.strip()
    try:
        star = main.find('span','ipc-rating-star ipc-rating-star--base ipc-rating-star--otherUserAlt review-rating').text.strip()
    except:
        star = "-"
    review = main.find('div','ipc-html-content ipc-html-content--base').text.strip()
    print("8. Top Review:", "\n", title, "\n", star, "\n", review)
    print()
    print('-'*90)

print()
print("TOP BOX OFFICE (US)")
full_html = driver.page_source
soup = BeautifulSoup(full_html,'html.parser')
month = soup.find('div','article listo').find('h4').text.strip()
print(month)
print()
print("="*90)
for i in range(1,11):
    movie_click = driver.find_element_by_xpath('//*[@id="boxoffice"]/table/tbody/tr[%s]/td[2]/a' %i)
    movie_click.click()
    time.sleep(1)
    get()
    driver.back()
    time.sleep(1)


TOP BOX OFFICE (US)
at Weekend of November 19 - 21, 2021



C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_5816/2529858144.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  movie_click = driver.find_element_by_xpath('//*[@id="boxoffice"]/table/tbody/tr[%s]/td[2]/a' %i)



# TOP: 1

1. Title: 고스트버스터즈 라이즈
2. IMDB Rating: 7.8/10
3. Summary: When a single mom and her two kids arrive in a small town, they begin to discover their connection to the original Ghostbusters and the secret legacy their grandfather left behind.
4. Director: Jason Reitman
5. Writers: Gil KenanJason ReitmanDan Aykroyd(based on: the 1984 "Ghostbusters", written by)
6. Stars: Carrie CoonPaul RuddFinn Wolfhard
7. Reviewers: 13
8. Top Review: 
 It's like fan fiction, but good fan fiction. 
 7/10 
 When a single mom and her two kids arrive in a small town, they begin to discover their connection to the original Ghostbusters and the secret legacy their grandfather left behind.

------------------------------------------------------------------------------------------

# TOP: 2

1. Title: 이터널스
2. IMDB Rating: 6.8/10
3. Summary: The saga of the Eternals, a race of immortal beings who lived on Earth and shaped its history and civilizations.
4. Director: Chloé Zhao
5. Writers: Chloé Zhao(scree

In [3]:
# 각 영화 별로 상단 3개의 리뷰만 추출
def get():
    print()
    print("# TOP:",x)
    print()
    full_html = driver.page_source
    soup = BeautifulSoup(full_html,'html.parser')
    main = soup.find('main','ipc-page-wrapper ipc-page-wrapper--base')
    print("1. Title:", main.find('h1','TitleHeader__TitleText-sc-1wu6n3d-0 dxSWFG').text.strip())
    print("2. IMDB Rating:", main.find('div','AggregateRatingButton__Rating-sc-1ll29m0-2 bmbYRW').text.strip())
    print("3. Summary:", main.find('span','GenresAndPlot__TextContainerBreakpointXL-cum89p-2 gCtawA').text.strip())
    makers = main.find_all('ul',class_="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt")
    print("4. Director:", makers[0].text.strip())
    print("5. Writers:", makers[1].text.strip())
    print("6. Stars:", makers[2].text.strip())
    click_review = driver.find_element_by_link_text('User reviews')
    click_review.click()
    full_html = driver.page_source
    soup = BeautifulSoup(full_html,'html.parser')
    print("7. Total Review Number:", soup.find('div','header').find('span').text.strip())
    print("8. Reviews:")
    box = soup.find('div','lister-list')
    title = box.find_all('a','title')
    name = box.find_all('span','display-name-link')
    review = box.find_all('div','content')
    time.sleep(2)
    for i in range(0,3):
        try:
            print("- Review", (i+1), ":", "\n", title[i].text.strip(), "\n", name[i].text.strip(), "\n", review[i].find('div').text.strip())
        except:
            print("- Review %s: No Review" %(i+1))
            pass
        print('-'*70)
        time.sleep(0.5)
    print()
    print('='*90)
    driver.back()

print()
print("TOP BOX OFFICE (US)")
full_html = driver.page_source
soup = BeautifulSoup(full_html,'html.parser')
month = soup.find('div','article listo').find('h4').text.strip()
print(month)
print()
print("="*90)
for x in range(1,11):
    movie_click = driver.find_element_by_xpath('//*[@id="boxoffice"]/table/tbody/tr[%s]/td[2]/a' %x)
    movie_click.click()
    time.sleep(1)
    get()
    driver.back()
    time.sleep(1)


TOP BOX OFFICE (US)
Weekend of November 19 - 21, 2021



C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14776/846616266.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  movie_click = driver.find_element_by_xpath('//*[@id="boxoffice"]/table/tbody/tr[%s]/td[2]/a' %x)



# TOP: 1

1. Title: 고스트버스터즈 라이즈
2. IMDB Rating: 7.8/10
3. Summary: When a single mom and her two kids arrive in a small town, they begin to discover their connection to the original Ghostbusters and the secret legacy their grandfather left behind.
4. Director: Jason Reitman
5. Writers: Gil KenanJason ReitmanDan Aykroyd(based on: the 1984 "Ghostbusters", written by)
6. Stars: Carrie CoonPaul RuddFinn Wolfhard


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14776/846616266.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  click_review = driver.find_element_by_link_text('User reviews')


7. Total Review Number: 545 Reviews
8. Reviews:
- Review 1 : 
 Surprisingly good. 
 Sleepin_Dragon 
 I'll be honest, after the last Ghostbusters movie, I wasn't keen on it, reluctantly I went along, and have to admit, I thoroughly enjoyed it. It feels like a worthy successor to the original film, it doesn't just ignore the past, it supports it. I would advise seeing the original movie ahead of this, as there is a lot of nostalgia surrounding it. I feel like they wanted it to respect the original, that it does.Some really nice special effects, a few perhaps not so good, but that's just me being picky. A good atmosphere, and a really good pace, there's nothing slow about the film, it doesn't lull, nor did it feel overlong. It's a bit goofy, it doesn't take itself too seriously.Great to see Dan Ackroyd, Bill Murray, and many other faces, if you're a fan of the original, you really will appreciate seeing them all here. Paul Rudd was awesome I thought, he stood out for me.
-----------------

...



# TOP: 7

1. Title: 007 노 타임 투 다이
2. IMDB Rating: 7.4/10
3. Summary: James Bond has left active service. His peace is short-lived when Felix Leiter, an old friend from the CIA, turns up asking for help, leading Bond onto the trail of a mysterious villain armed with dangerous new technology.
4. Director: Cary Joji Fukunaga
5. Writers: Neal Purvis(screenplay by)Robert Wade(screenplay by)Cary Joji Fukunaga(screenplay by)
6. Stars: Daniel CraigAna de ArmasRami Malek
7. Total Review Number: 2,995 Reviews
8. Reviews:
- Review 1 : 
 golden hour Bond 
 SnoopyStyle 
 James Bond (Daniel Craig) is married to Madeleine Swann (Léa Seydoux). She has a secret past. He's visiting the grave of Vesper Lynd when Spectre attacks. Blofeld claims that Madeleine is a daughter of Spectre. It's five years later. Bond is retired and living alone. Russian bio-weapon scientist Valdo Obruchev has been kidnapped by new baddie Lyutsifer Safin (Rami Malek). MI6 and the CIA are at cross-purposes. Both are searching 

- Review 3 : 
 No Time To Die 
 Prismark10 
 Bond is back at last. Daniel Craig is the longest serving Bond with the fewest films.Once again Bond is out of the service, bumming around the Caribbean.It's Felix Leiter who coaxes him back to action. Only to find that M has replaced him with a new 007.The target is a biological weapon that horns in on DNA. The British were developing it but lost it to Spectre, or so they thought.For Bond it's a reunion with Blofeld and Madeleine, a woman he thought he loved but let go, mistakenly. She never did have time to tell him her secret.However there is a new foe called Safin and he is bearing a grudge but it is against Spectre.The movie is long and lacks a spectacular action sequence. It also feels a lot like a Christopher Nolan film. Just check out all those black Range Rovers, the booming bass and the car action scenes.I also thought Safin himself was a weak baddie. I could understand his hatred for Blofeld but not against the world or Bond.The s

- Review 3 : 
 Timothée Chalamet gets his annus mirabilis in Wes Anderson's ode to print journalism 
 lee_eisenberg 
 I've liked every Wes Anderson movie that I've seen, and to that I can now add "The French Dispatch". If you've seen Anderson's previous movies, then you should have an idea of what to expect here (centered scenes, quirky characters, clever dialogue). In this case, the plot involves the final issue of a magazine and the topics that the magazine will cover. Lots of neat stuff in store.As always, Anderson casts Bill Murray, while his occasional cast member Anjelica Huston narrates. Also appearing are Anderson regulars Owen Wilson, Jason Schwartzmann, Adrien Brody, Bob Balaban, Willem Dafoe, Edward Norton, Frances McDormand and Tilda Swinton. Appearing for the first time in an Anderson movie are Timothée Chalamet, Henry Winkler, Lois Smith, Elisabeth Moss, Griffin Dunne, Benicio Del Toro, Christoph Waltz, Jeffrey Wright and Liev Schreiber (if I remember right, Mathieu Amalr

- Review 3 : 
 One of the best of the year and of Kenneth Branagh's life. 
 jdesando 
 "Go. Go now. Don't look back. I love you, son." Granny (Judi Dench)You can complain that Kenneth Branagh his filtered his 9-year-old Buddy (Jude Hill) through his own rose-colored revery of the 1969 bloody ethno-nationalist uprising in Belfast, and you'd be right. However, like all of us remembering, that past is most pleasantly remembered through the lens of loving family struggle that binds.While Branagh doesn't shy away from how the Northern Ireland Troubles between Protestants and Catholics was challenging all families, his endearing portrait of Buddy as a curious and sweet, albeit precocious, school boy for whom the biggest conflicts are figuring out how not to emigrate from Belfast because of the violence and connecting with the elusive little blonde who occupies the top of her class with Buddy.One of the best movies of the year, Belfast gives scant references to Branagh's eventual rise to the 

- Review 2 : 
 Engaging and comical kids story for adults 
 CinemaSerf 
 I'm going to start my review with a moan. Since when was it OK for parents to just let their young children chat throughout a film at a cinema? Or to wander about the aisles dropping popcorn everywhere? My parents would have had a fit, and we would have been unceremoniously removed never to return. Cinema is a shared experience on a big screen, not a surrogate baby-sitting service! Phew, now I have got that off my chest, this is quite a thought-provoking animation. The premiss is nothing especially new, a toy that every kid must have (remember the "Cabbage Patch Doll?") but this story has a far more sinister undertone to it. "Marc" is the bright-eyed, enthusiastic, young CEO of "Bubble" and in best Steve Jobs tradition, addresses his huge team from the "launchatorium" announcing their newest product - the "B-bot". A small, egg-shaped, gadget that is to become the best friend of every child. It will befriend them, 